# Evaluation Script
Normally you would generate a larger sample of examples for evaluation and create a pipeline checking for syntactically and/or factual correctness. In this case the fine-tuned models didn't perform as expected so we stayed with manual evaluation based on some hand-crafted examples.

In [1]:
from transformers import GenerationConfig
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define German and English Evaluation Samples and Models

In [2]:
import pandas as pd

german_test_data_dicts = [
    {
        "function_name": "create_calendar_entry",
        "query": "Erstelle einen neuen Kalendertermin für ein Weihnachtsessen bei meinen Eltern am 24.12.2024 um 18 Uhr für 3 Stunden",
        "parameters": '<oc_1>("Weihnachtsessen bei meinen Eltern", "2024-12-24", "18:00", 180)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Erstelle einen Termin für einen Arzttermin am 13.6.2023 um 12 Uhr für 60 Minuten",
        "parameters": '<oc_1>("Arzttermin", "2023-06-13", "12:00", 60)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Neuen Kalendertermin für Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten anlegen",
        "parameters": '<oc_1>("Projektpräsentation", "2024-06-24", "18:00", 90)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Lege einen Termin für ein Teamabendessen im Restaurant am 4. Juli 2024 ab 20 Uhr für 1,5 Stunden an",
        "parameters": '<oc_1>("Teamabendessen im Restaurant", "2024-07-04", "20:00", 90)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Plane einen Termin für einen Kundengespräch am 16. Februar 2025 um 15:00 für 2 Stunden ein",
        "parameters": '<oc_1>("Kundengespräch", "2025-02-16", "15:00", 120)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Lege einen Termin für ein einstündiges Info-Event am 27. Januar 2026 um 10:00 Uhr an",
        "parameters": '<oc_1>("Info-Event", "2024-01-27", "10:00", 60)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Erstelle einen neuen Kalendartermin für ein zweistündiges Rockkonzert auf der Kieler Woche am 26. Juni 2024 um 21:00 Uhr",
        "parameters": '<oc_1>("Rockkonzert auf der Kieler Woche", "2024-06-26", "21:00", 120)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Neuen Kalendareintrag für eine eineinhalbstündige Bahnfahrt nach Hamburg am 06.11.2023 um 6 Uhr erstellen",
        "parameters": '<oc_1>("Bahnfahrt nach Hamburg", "2023-11-06", "06:00", 90)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Erstelle Kalendareintrag für dreistündigen Workshop am 03.03.2026 um 11 Uhr ",
        "parameters": '<oc_1>("Workshop", "2026-03-03", "11:00", 180)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Lege einen sechsstündigen Termin für eine Silvesterparty am 31.12.2025 ab 20 Uhr an",
        "parameters": '<oc_1>("Silvesterparty", "2025-12-31", "20:00", 360)<oc_end>'
    }
]

In [3]:
english_test_data_dicts = [
    {
        "function_name": "create_calendar_entry",
        "query": "Create a new calendar appointment for a Christmas dinner at my parents' house on 24.12.2024 at 6 pm for 3 hours",
        "parameters": '<oc_1>("Christmas dinner with my parents", "2024-12-24", "18:00", 180)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create an appointment for a doctor's appointment on 13.6.2023 at 12 noon for 60 minutes",
        "parameters": '<oc_1>("Doctor appointment", "2023-06-13", "12:00", 60)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create new calendar appointment for project presentation on 24.06.2024 at 6 pm for 90 minutes",
        "parameters": '<oc_1>("Project presentation", "2024-06-24", "18:00", 90)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create an appointment for a team dinner in the restaurant on July 4, 2024 from 8 p.m. for 1.5 hours",
        "parameters": '<oc_1>("Team dinner at the restaurant", "2024-07-04", "20:00", 90)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Schedule an appointment for a customer meeting on February 16, 2025 at 15:00 for 2 hours",
        "parameters": '<oc_1>("customer meeting", "2025-02-16", "15:00", 120)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create an appointment for a one-hour info event on January 27, 2026 at 10:00 a.m.",
        "parameters": '<oc_1>("Info event", "2024-01-27", "10:00", 60)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create a new calendar date for a two-hour rock concert at Kieler Woche on June 26, 2024 at 21:00",
        "parameters": '<oc_1>("Rock concert at Kieler Woche", "2024-06-26", "21:00", 120)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create new calendar entry for a one and a half hour train journey to Hamburg on 06.11.2023 at 6 a.m.",
        "parameters": '<oc_1>("Train journey to Hamburg", "2023-11-06", "06:00", 90)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create calendar entry for three-hour workshop on 03.03.2026 at 11 am ",
        "parameters": '<oc_1>("Workshop", "2026-03-03", "11:00", 180)<oc_end>'
    },
    {
        "function_name": "create_calendar_entry",
        "query": "Create a six-hour appointment for a New Year's Eve party on 31.12.2025 from 8 p.m.",
        "parameters": '<oc_1>("New Years Eve party", "2025-12-31", "20:00", 360)<oc_end>'
    }
]

In [4]:
english_models = [
    "english-function-calling-model",
    "english-function-calling-model-0.5b",
    "phi-3-english-function-calling-model"
]

In [5]:
german_models = [
    "german-function-calling-model",
    "german-function-calling-model-0.5b",
    "german-function-calling-model-3-epochs",
    "german-function-calling-orpo-0.5b-model",
    "german-function-calling-orpo-model",
]

### Evaluate the German Models

In [6]:
for german_model in german_models:
    torch.cuda.empty_cache()
    
    tokenizer = AutoTokenizer.from_pretrained(german_model)
    model = AutoModelForCausalLM.from_pretrained(german_model)
    model.to("cuda:0")

    early_stopping_token = "<oc_end>"
    eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
    eos_token_id

    generation_config = GenerationConfig(
        penalty_alpha=0.0,
        do_sample=True,
        top_k=1,
        temperature=0.1,
        repetition_penalty=2.0,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=eos_token_id
    )

    for german_test_data_dict in german_test_data_dicts:
        print(german_test_data_dict["query"])
        prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
        Befehl: {query}
        Antwort: """.format(query=german_test_data_dict["query"])

        # Tokenize the prompt and get the input tensor
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        input_length = inputs['input_ids'].shape[1]  # Get the length of input tokens

        # Generate the output tokens
        outputs = model.generate(**inputs, generation_config=generation_config)

        # Slice the output tokens to exclude the input tokens
        generated_tokens = outputs[0, input_length:]

        # Decode and print the generated tokens
        generated_parameters = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        german_test_data_dict["generated_parameters" + "_" + german_model] = generated_parameters
        print(generated_parameters)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:32<00:00, 16.48s/it]


Erstelle einen neuen Kalendertermin für ein Weihnachtsessen bei meinen Eltern am 24.12.2024 um 18 Uhr für 3 Stunden
<oc_1>("Weihnachtssuch", "2025-12-24", "18:00", 18)<oc_end>
Erstelle einen Termin für einen Arzttermin am 13.6.2023 um 12 Uhr für 60 Minuten
<oc_1>("Arzert", "2024-05-13", "12:00", 60)<oc_end>
Neuen Kalendertermin für Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten anlegen
<oc_1>("Projektpräsentations", "2025-06-24", "18:00", 90)<oc_end>
Lege einen Termin für ein Teamabendessen im Restaurant am 4. Juli 2024 ab 20 Uhr für 1,5 Stunden an
<oc_1>("Restaurantbesuch", "2026-07-3")<oc_end>
Plane einen Termin für einen Kundengespräch am 16. Februar 2025 um 15:00 für 2 Stunden ein
<oc_1>("Konzert", "2024-02-16", "17:00", 12)<oc_end>
Lege einen Termin für ein einstündiges Info-Event am 27. Januar 2026 um 10:00 Uhr an
<oc_1>("Infoveranstaltung Studium", "2025-01-28", "10:00", 6)<oc_end>
Erstelle einen neuen Kalendartermin für ein zweistündiges Rockkonzert auf der Kieler W

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Erstelle einen neuen Kalendertermin für ein Weihnachtsessen bei meinen Eltern am 24.12.2024 um 18 Uhr für 3 Stunden
<oc_1>("Weihn ccapt", "2025-12-24","One十八")<oc_end>
Erstelle einen Termin für einen Arzttermin am 13.6.2023 um 12 Uhr für 60 Minuten
<oc_1>("Arzitterd8c57", "2024-09-13","One十二")<oc_end>
Neuen Kalendertermin für Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten anlegen
<oc_1>("Projektpräsentation", "2023-05 -24', '18':00", 90)<oc_end>
Lege einen Termin für ein Teamabendessen im Restaurant am 4. Juli 2024 ab 20 Uhr für 1,5 Stunden an
<oc_1>("Restaurantbesuch", "2026-07-4", "@20)<oc_end>
Plane einen Termin für einen Kundengespräch am 16. Februar 2025 um 15:00 für 2 Stunden ein
<oc_1>("Kundereinträge", "2023-04-16", "$15:00$",)<oc_end>
Lege einen Termin für ein einstündiges Info-Event am 27. Januar 2026 um 10:00 Uhr an
<oc_1>("Info-Meeting", "2025 -09-28", "$10:40$", 60)<oc_end>
Erstelle einen neuen Kalendartermin für ein zweistündiges Rockkonzert auf der Kieler Woc

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:31<00:00, 15.82s/it]


Erstelle einen neuen Kalendertermin für ein Weihnachtsessen bei meinen Eltern am 24.12.2024 um 18 Uhr für 3 Stunden
<oc_1>("Weihnachtsessen Bei meinem Elernzeit", "2025-12-26", "18:00", -)<oc_end>
Erstelle einen Termin für einen Arzttermin am 13.6.2023 um 12 Uhr für 60 Minuten
<oc_1>("Arzeit in den Kalendar", "2025-06-14", "12:00", 80)<oc_end>
Neuen Kalendertermin für Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten anlegen
<oc_1>("Projektpräsentation", "2025-06-34", "18:00", 70)<oc_end>
Lege einen Termin für ein Teamabendessen im Restaurant am 4. Juli 2024 ab 20 Uhr für 1,5 Stunden an
<oc_1>("Restaurantbesuch", "2023-07-49")<oc_end>
Plane einen Termin für einen Kundengespräch am 16. Februar 2025 um 15:00 für 2 Stunden ein
<oc_1>("title", date)<oc_end>
Lege einen Termin für ein einstündiges Info-Event am 27. Januar 2026 um 10:00 Uhr an
<oc_1>("Infoveranstaltung Studium", "2025-09-37)<oc_end>
Erstelle einen neuen Kalendartermin für ein zweistündiges Rockkonzert auf der Kieler 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Erstelle einen neuen Kalendertermin für ein Weihnachtsessen bei meinen Eltern am 24.12.2024 um 18 Uhr für 3 Stunden
<oc_1>("Weihnachtessen bei meinem an", "2024-12-24", '18:05', 96)<oc_end>
Erstelle einen Termin für einen Arzttermin am 13.6.2023 um 12 Uhr für 60 Minuten
<oc_1>("Arzert", "2023-05-14", '18c97e12d8")<oc_end>
Neuen Kalendertermin für Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten anlegen
<oc_1>("Projektvortrag", "2024-05-34", '18:00', 70)<oc_end>
Lege einen Termin für ein Teamabendessen im Restaurant am 4. Juli 2024 ab 20 Uhr für 1,5 Stunden an
<oc_1>("Restaurantbesuch", "title (str): Der Titel des Kalender-API.

Rückgabe:
bool: True wenn der Aufruf der Kalenders-API erfolgreich war, andernfalls False.

"""Antwort: <oc_3>("Restaurantbesuch", '2024-07-
Plane einen Termin für einen Kundengespräch am 16. Februar 2025 um 15:00 für 2 Stunden ein
<oc_1>("Konzert", "title (str): Der Titel des Kalendereintrags.

Rückgabe:
bool: True wenn der Aufruf der Kalender-API erf

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.83s/it]


Erstelle einen neuen Kalendertermin für ein Weihnachtsessen bei meinen Eltern am 24.12.2024 um 18 Uhr für 3 Stunden
<oc_1>("Websites", "2024-12-24", "18:00", 180)<oc_end>
Erstelle einen Termin für einen Arzttermin am 13.6.2023 um 12 Uhr für 60 Minuten
<oc_1>("Arabd Termine", "YYYY-MM-DD9e")<oc_end>
Neuen Kalendertermin für Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten anlegen
<oc_3>("Projektpräsentation", "YYYY-MM-DD")<oc_end>
Lege einen Termin für ein Teamabendessen im Restaurant am 4. Juli 2024 ab 20 Uhr für 1,5 Stunden an
<oc_3>("Teamabe", "YYYY-MM-DD")<oc_end>
Plane einen Termin für einen Kundengespräch am 16. Februar 2025 um 15:00 für 2 Stunden ein
<oc_1>("Kundeneintrag fa8fde9c4d")<oc_end>
Lege einen Termin für ein einstündiges Info-Event am 27. Januar 2026 um 10:00 Uhr an
<oc_3>("Infoveranstaltung Studium", "YYYY-MM-DD9e")<oc_end>
Erstelle einen neuen Kalendartermin für ein zweistündiges Rockkonzert auf der Kieler Woche am 26. Juni 2024 um 21:00 Uhr
<oc_1>("RockKonz

In [7]:
german_test_data_dicts

[{'function_name': 'create_calendar_entry',
  'query': 'Erstelle einen neuen Kalendertermin für ein Weihnachtsessen bei meinen Eltern am 24.12.2024 um 18 Uhr für 3 Stunden',
  'parameters': '<oc_1>("Weihnachtsessen bei meinen Eltern", "2024-12-24", "18:00", 180)<oc_end>',
  'generated_parameters_german-function-calling-model': '<oc_1>("Weihnachtssuch", "2025-12-24", "18:00", 18)<oc_end>',
  'generated_parameters_german-function-calling-model-0.5b': '<oc_1>("Weihn ccapt", "2025-12-24","One十八")<oc_end>',
  'generated_parameters_german-function-calling-model-3-epochs': '<oc_1>("Weihnachtsessen Bei meinem Elernzeit", "2025-12-26", "18:00", -)<oc_end>',
  'generated_parameters_german-function-calling-orpo-0.5b-model': '<oc_1>("Weihnachtessen bei meinem an", "2024-12-24", \'18:05\', 96)<oc_end>',
  'generated_parameters_german-function-calling-orpo-model': '<oc_1>("Websites", "2024-12-24", "18:00", 180)<oc_end>'},
 {'function_name': 'create_calendar_entry',
  'query': 'Erstelle einen Termin 

### Evaluate the English Models

In [8]:
for english_model in english_models + german_models:
    torch.cuda.empty_cache()
    
    tokenizer = AutoTokenizer.from_pretrained(english_model)
    model = AutoModelForCausalLM.from_pretrained(english_model)
    model.to("cuda:0")

    early_stopping_token = "<oc_end>"
    eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
    eos_token_id

    generation_config = GenerationConfig(
        penalty_alpha=0.0,
        do_sample=True,
        top_k=1,
        temperature=0.1,
        repetition_penalty=2.0,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=eos_token_id
    )

    for english_test_data_dict in english_test_data_dicts:
        print(english_test_data_dict["query"])
        prompt = """Below is the query from the users, please choose the correct function and generate the parameters to call the function.
        Query: {query}
        Response: """.format(query=english_test_data_dict["query"])

        # Tokenize the prompt and get the input tensor
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        input_length = inputs['input_ids'].shape[1]  # Get the length of input tokens

        # Generate the output tokens
        outputs = model.generate(**inputs, generation_config=generation_config)

        # Slice the output tokens to exclude the input tokens
        generated_tokens = outputs[0, input_length:]

        # Decode and print the generated tokens
        generated_parameters = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        english_test_data_dict["generated_parameters" + "_" + english_model] = generated_parameters
        print(generated_parameters)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.24s/it]


Create a new calendar appointment for a Christmas dinner at my parents' house on 24.12.2024 at 6 pm for 3 hours
<oc_1>("Christmas Dinner", "2025-12-24", '07:00', 180)<oc_end>
Create an appointment for a doctor's appointment on 13.6.2023 at 12 noon for 60 minutes
<oc_1>("Doctor", "2023-05-14", "12 Noon", 60)<oc_end>
Create new calendar appointment for project presentation on 24.06.2024 at 6 pm for 90 minutes
<oc_1>("Project presentation", "2025-13-24", "07:00", 8)<oc_end>
Create an appointment for a team dinner in the restaurant on July 4, 2024 from 8 p.m. for 1.5 hours
<oc_1>("Team Dinner", "2023-07-4", '09:00', 60)<oc_end>
Schedule an appointment for a customer meeting on February 16, 2025 at 15:00 for 2 hours
<oc_1>("Customer Meeting", "2024-03-17", "18:00", 12)<oc_end>
Create an appointment for a one-hour info event on January 27, 2026 at 10:00 a.m.
<oc_1>("Study information events", "2025-09-34", '10:00a.m")<oc_end>
Create a new calendar date for a two-hour rock concert at Kieler W

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Create a new calendar appointment for a Christmas dinner at my parents' house on 24.12.2024 at 6 pm for 3 hours
<oc_1>("Christmas Dinner with My Parents", "2024-12-24", '08':"05":""(7)")<oc_end>
Create an appointment for a doctor's appointment on 13.6.2023 at 12 noon for 60 minutes
<oc_1>("Doctor’s calendar", "2023- June -14","15:00", 6)<oc_end>
Create new calendar appointment for project presentation on 24.06.2024 at 6 pm for 90 minutes
<oc_1>("Project presentations", "2023- June -15": '08':"07:pm","6")<oc_end>
Create an appointment for a team dinner in the restaurant on July 4, 2024 from 8 p.m. for 1.5 hours
<oc_1>("Team Dinner", "2026-07-09", '08': True)<oc_end>
Schedule an appointment for a customer meeting on February 16, 2025 at 15:00 for 2 hours
<oc_1>("Weekly team Meeting", "2024-13-07", '15:00', 18)<oc_end>
Create an appointment for a one-hour info event on January 27, 2026 at 10:00 a.m.
<oc_1>("Info events", "2024-03-09", 'Tten': -15)<oc_end>
Create a new calendar date for a 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Create a new calendar appointment for a Christmas dinner at my parents' house on 24.12.2024 at 6 pm for 3 hours
<oc_1>("aChristmasdbee", "2025-12-27", "06:00", -8)<oc_end>
Create an appointment for a doctor's appointment on 13.6.2023 at 12 noon for 60 minutes
<oc_1>("a Doctor’s appointments", "2025-06-14", "12noon")<oc_end>
Create new calendar appointment for project presentation on 24.06.2024 at 6 pm for 90 minutes
<oc_1>("Projektpräsentation", "2025-06-31", "06:00", 7)<oc_end>
Create an appointment for a team dinner in the restaurant on July 4, 2024 from 8 p.m. for 1.5 hours
<oc_1>("Restaurantbesuch", "2023-07-09")<oc_end>
Schedule an appointment for a customer meeting on February 16, 2025 at 15:00 for 2 hours
<oc_1>("Restaurantbesuch", "2024-08-30", False)<oc_end>
Create an appointment for a one-hour info event on January 27, 2026 at 10:00 a.m.
<oc_1>("Infoevent", "2024-03-29")<oc_end>
Create a new calendar date for a two-hour rock concert at Kieler Woche on June 26, 2024 at 21:00
<

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<oc_1>("a6e7c9d41")<oc_end>
Create a new calendar appointment for a Christmas dinner at my parents' house on 24.12.2024 at 6 pm for 3 hours
<oc_1>("achentliches Teammeeting", "2024-18-25",help)<oc_end>
Create an appointment for a doctor's appointment on 13.6.2023 at 12 noon for 60 minutes
<oc_1>("adoctorsappointment", "2023-05-14", '18 Uhr', 60)<oc_end>
Create new calendar appointment for project presentation on 24.06.2024 at 6 pm for 90 minutes
<oc_1>("Projektpräsentation", "2024-05-31", '06': Uhr für ein eineinhalbstündiges wöchentliches Team-Meeting am 28. April 2024 um 7 Uhr mit einer Dauer von 1
Create an appointment for a team dinner in the restaurant on July 4, 2024 from 8 p.m. for 1.5 hours
<oc_1>("a7b9d6adions", False)<oc_end>
Schedule an appointment for a customer meeting on February 16, 2025 at 15:00 for 2 hours
<oc_1>("appointment", "2025-03-14",help897b")<oc_end>
Create an appointment for a one-hour info event on January 27, 2026 at 10:00 a.m.
<oc_1>("Infoveranstaltung Stu

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:30<00:00, 15.46s/it]


Create a new calendar appointment for a Christmas dinner at my parents' house on 24.12.2024 at 6 pm for 3 hours
<oc_1>("Christmas Dinner", "YYYY-MM-DD9")<oc_end>
Create an appointment for a doctor's appointment on 13.6.2023 at 12 noon for 60 minutes
<oc_1>("Vorlesung", "2023- Juni-13")<oc_end>
Create new calendar appointment for project presentation on 24.06.2024 at 6 pm for 90 minutes
<oc_1>("Projektpräsentation", "YYYY-MM-DD1")<oc_end>
Create an appointment for a team dinner in the restaurant on July 4, 2024 from 8 p.m. for 1.5 hours
<oc_1>("Teammeeting", "YYYY-MM-DD9")<oc_end>
Schedule an appointment for a customer meeting on February 16, 2025 at 15:00 for 2 hours
<oc_1>("Customer", "YYYY-MM-DD9e")<oc_end>
Create an appointment for a one-hour info event on January 27, 2026 at 10:00 a.m.
<oc_1>("Infoevent", "YYYY-MM-DD9")<oc_end>
Create a new calendar date for a two-hour rock concert at Kieler Woche on June 26, 2024 at 21:00
<oc_1>("Rockkonzert", "YYYY-MM-DD9b")<oc_end>
Create new ca

In [9]:
english_test_data_dicts

[{'function_name': 'create_calendar_entry',
  'query': "Create a new calendar appointment for a Christmas dinner at my parents' house on 24.12.2024 at 6 pm for 3 hours",
  'parameters': '<oc_1>("Christmas dinner with my parents", "2024-12-24", "18:00", 180)<oc_end>',
  'generated_parameters_english-function-calling-model': '<oc_1>("Christmas Dinner", "2025-12-24", \'07:00\', 180)<oc_end>',
  'generated_parameters_english-function-calling-model-0.5b': '<oc_1>("Christmas Dinner with My Parents", "2024-12-24", \'08\':"05":""(7)")<oc_end>',
  'generated_parameters_phi-3-english-function-calling-model': '<oc_1>("Christmas time with me", "2024-12-25", \'09:MM8")<oc_end>\nFunction description: def create_calender entry(title, date, only_not_finished=False):\n"""\nCreate A one (',
  'generated_parameters_german-function-calling-model': '<oc_1>("Christmasessen bei meinen Eltern", "2025-12-24", "06:00", 8)<oc_end>',
  'generated_parameters_german-function-calling-model-0.5b': '<oc_1>("aChristmas

### Save the Results

In [10]:
german_results = pd.DataFrame(german_test_data_dicts)
german_results.to_csv("german-results.csv", sep=";", index=False)

In [11]:
english_results = pd.DataFrame(english_test_data_dicts)
english_results.to_csv("english-results.csv", sep=";", index=False)